In [4]:
# Lo primero sera importar las librerias y estableces algunas configuraciones para
# facilitar la lectura en pantalla

import pandas as pd
import numpy as np
import re
import unicodedata
import warnings
import operator
import nltk
nltk.download('cess_esp')
nltk.download('stopwords')
from nltk.corpus import cess_esp as cess
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

from nltk.stem import WordNetLemmatizer

from nltk.corpus import stopwords
sw = stopwords.words('spanish')

import difflib
import platform

if platform.system()=='Linux':
    path_datos = '/home/luis/mensajes.data/'
elif platform.system()=='Windows':
    path_datos = 'C:/Users/Luis/Documents/Ciencia de Datos/Dataset1/'


warnings.filterwarnings('ignore')
pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)
pd.options.display.max_rows = 4000

[nltk_data] Downloading package cess_esp to /home/luis/nltk_data...
[nltk_data]   Package cess_esp is already up-to-date!
[nltk_data] Downloading package stopwords to /home/luis/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [43]:
# leer los datos y cargarlos en dataframe, validando la carga al presentar informacion basica
# del archivo
mensajes = pd.read_json(path_datos+'mensajespd.json')

print(mensajes.shape)

# borramos registros duplicados desde los datos originales
mensajes.drop_duplicates(inplace=True)

#print(mensajes.head())
print("\n")
print("Cantidad de registros de mensajes = %3d" % mensajes.shape[0])
print("\n")
print(mensajes.columns)

# ordenamos convenientemente por telef origen  y fecha+hora de envio
mensajes.sort_values(by=["Enviado/por", "Fecha/Hora"])

# reacomodando indices
indice = list(range(mensajes.shape[0]))
mensajes.index = indice



(1197, 3)


Cantidad de registros de mensajes = 1101


Index(['Enviado/por', 'Fecha/Hora', 'Texto'], dtype='object')


**A continuacion seran creadas y calculadas las caracteristicas de interes**

In [44]:

# convertimos a formato de fecha la columna "Fecha/Hora"
mensajes["Fecha/Hora"] = pd.to_datetime(mensajes["Fecha/Hora"])

# creamos variable de interes tiempo en segundos (hora del dia en segundos)

mensajes['ts'] = mensajes["Fecha/Hora"].apply(lambda x: x.hour*60*60+x.minute*60+x.second)

# creamos variable el tiempo entre mensajes: tem
# voala! funciona.. 

mensajes["tem"] = np.zeros(mensajes.shape[0], dtype=int)

mensajes["tem"] = abs(mensajes["ts"].diff())

# y creamos dos nuevas variables, una que indique si "Texto" proviene de un suscriptor del servicio
# de internet (0) o no (1) y otra que partiendo del contenido de "Texto" clasifique en mensaje,
# lo cual se hara luego de analizar la variable "Texto" mas adelante

# por defecto ninguno es suscriptor
mensajes["es_suscriptor"] = np.full(mensajes.shape[0], 9)

# aun no se a establecido criterio para clasificar, se inicializa en 0
mensajes["tipo_mensaje"] = np.full(mensajes.shape[0], 9)
# donde: 0->usuario reporta falla, 1->usuario solicita informacio cuota del servicio,
# 2->todo lo demas (no suscriptores)


**el dataset requiere depuracion ya que multiples
 registros (algunos), contienen informacion de un solo mensaje, lo que distorciona la
 interpretacion de dichos mensajes**

In [45]:
%%time
# con este arreglo de indice calculado a continuacion, concatenamos 
# mensajes divididos en la fila i-1 y posterior eliminamos los (i) ndice
# ya q seran redundantes
# hay q tener en cuenta filtrar q mensajes a concatenar sean del mismo
# sender y que contenido de mensajes sean distintos

# cosideramos solo aquellos mensajes sucesivos que hayan sido recibidos
# en un intervalo de tiempo menor a 10 segundos ya que en este subgrupo se encuentran
# los mensajes divididos


indimulmensajes = mensajes[mensajes["tem"]<10].index

# uniendo y borrando registros redundantes, despues de unir mensajes fragmentados.
# Hay q recorrer el dataset de abajo hacia arriba (reverse) dado que: puede haber 
# mensajes fraccionados en mas de 2 partes y a la vez es posible borrar cada
# registro redundante una vez que se unan.


for i in reversed(indimulmensajes):
    if operator.and_(mensajes["Enviado/por"][i-1]==mensajes["Enviado/por"][i], mensajes["Texto"][i-1]!=mensajes["Texto"][i]):
        mensajes["Texto"][i-1] += mensajes["Texto"][i]
        mensajes.drop(mensajes.index[i], inplace = True)


# reacomodando indices
indice = list(range(mensajes.shape[0]))
mensajes.index = indice
        
# aqui se eliminan mensajes que contengan 2 o menos palabras


var1 = mensajes.index[mensajes["Texto"].apply(lambda x: len(tokenizer.tokenize(x))<3) ]

mensajes.drop(mensajes.index[var1], inplace = True)

# reacomodando indices
indice = list(range(mensajes.shape[0]))
mensajes.index = indice



Wall time: 15.9 s


In [46]:
# ahora se va a proceder a colocar en minuscula, todo el contenido de la variable "Texto"
# todo a minuscula

mensajes.Texto = mensajes.Texto.str.lower()

# y luego eliminamos todos los acentos 


def strip_accents(text):
    """
    Strip accents from input String.

    :param text: The input string.
    :type text: String.

    :returns: The processed String.
    :rtype: String.
    """
    try:
        text = unicode(text, 'utf-8')
    except (TypeError, NameError): # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)

mensajes["Texto"] = mensajes["Texto"].apply(lambda tx: strip_accents(tx))

**A partir de aqui comienza la clasificacion**

In [47]:
# leer telefonos conocidos de clientes desde archivo
# al archivo tlfclientes.csv se le agrego un caracter alfanumerico al comienzo
# de cada registro para que pandas leyera como texto y no como numero

telef_clientes = pd.read_csv(path_datos+'tlfclientes.csv')

# ahora se convierte telef_clientes.telef eliminando los caracteres- al inicio y final
telef_clientes["telef"] = telef_clientes["telef"].str.replace("-","")

# y se carga a memoria como lista, ya q son pocos
lista_telef = telef_clientes["telef"].tolist()

# ahora clasificamos como 0 la variable "es_suscriptor", si coincide lista con 
# "Enviado/por" del dataframe

mensajes.loc[mensajes["Enviado/por"].isin(lista_telef), "es_suscriptor"] = 0 


# mensajes de origen conocido que pueden clasificarse 
tlf_fami =['04140819214', '04147227018', '04261348023', '04140809688', '04164725484']

mensajes.loc[mensajes["Enviado/por"].isin(tlf_fami), "es_suscriptor"] = 1 
mensajes.loc[mensajes["Enviado/por"].isin(tlf_fami), "tipo_mensaje"] = 2 


# este bloque son mensajes spam, de publicidad
mensajes.loc[mensajes['Enviado/por'].str.len()<11, "es_suscriptor" ] = 1

mensajes.loc[mensajes['Enviado/por'].str.len()<11, "tipo_mensaje" ] = 2

# guardamos todo el nuevo dataframe con datos limpios, preprocesados
mensajes.to_pickle(path_datos+"mensajes.pkl")


** Funciones que se usaran en preprocesamiento**

In [1]:
tokenizer = RegexpTokenizer(r'\w+')
lemmatizer = WordNetLemmatizer()

def posicion_palabra(palabra, tokens):
    pos_palabra = -1
    if len(palabra)<=3:
        if palabra in tokens:
            pos_palabra = tokens.index(palabra)
    else:
        var = difflib.get_close_matches(palabra, tokens, n=1, cutoff=0.8)
        if len(var)>0:
            pos_palabra = tokens.index(var[0])
    return pos_palabra

def busca_palabra(a, b, texto):
    tokens = tokenizer.tokenize(texto)
    [lemmatizer.lemmatize(token) for token in tokens]
    pos_a = posicion_palabra(a, tokens)
    pos_b = posicion_palabra(b, tokens)
    
    if pos_a < 0 or pos_b <0 : resultado = False
    elif len(a)<=3: resultado = pos_a < pos_b
    else: resultado = True
    
    return resultado
    
def recore_pares(conten_par, texto):
    resultado = False
    for conte in conten_par:
        a = conte[0]
        b = conte[1]
        if busca_palabra(a, b, texto):
            resultado = True
            break
            
    return resultado
    
def form_pares(cadens1, cadens2):
    pares = []
    for pal1 in cadens1: 
        for pal2 in cadens2: 
            pares.append([pal1, pal2])
    return pares
    

def frecuencia(colum, label):
    frec = pd.crosstab(index=colum, columns = "frecuencia")
    frec = frec.rename(index=label)
    return frec    
    
def tabcruz(v_filas, v_columnas, et_filas, et_columnas ):
    tacr = pd.crosstab(index = v_filas, columns = v_columnas, margins = True, margins_name = "Total" )
    tacr = tacr.rename(index=et_filas) 
    tacr = tacr.rename(columns=et_columnas)
    return tacr

   
# definicion de etiquetas de valores de caracteristicas
eti_suscriptor = {0: 'Si', 1: 'No', 9: 'Sin Clase'}
eti_tmensaje = {0: 'falla Serv', 1: 'Info Pago', 2: 'Otros', 9: 'Sin Clase'}




**Clasificacion para reporte falla de servicio**

In [49]:
# palabras de uso comun de los suscriptores, para reportar falla del servicio de internet    
palab_clave1 = ['no', 'sin', 'falla', 'caida']

palab_clave2 = ['internet', 'conexion', 'red', 'servicio', 'wifi', 'senal']

# formamos pares de palabras claves
palab_clave = form_pares(palab_clave1, palab_clave2)

var1 = mensajes.index[mensajes["Texto"].apply(lambda x: recore_pares(palab_clave, x))]

# aqui listado para depurar var1 
# generar celda para ejecutar la siguiente instruccion:
# mensajes.loc[mensajes.index[var1]]


In [50]:
# obtenido manualmente comparando estos registros no pertenecen a esta categoria
var2 = [6, 209, 554]

var1 = sorted(list(set(var1) - set(var2)))

# clasificamos los mensajes
mensajes.loc[mensajes.index[var1], "tipo_mensaje" ] = 0
mensajes.loc[mensajes.index[var1], "es_suscriptor" ] = 0


**Clasificacion para reporte pago, solicitus informacion de pago

In [51]:
# mensaje de texto para considerarlo como categoria 1 
# funciona, hay que variar el cutoff para aumentar o disminuir la presicion
# en spyder se obtienen buenos resultados ajustando el valor a 0.7 pero
# aca se debe ajustar a 0.8 para reducir falsos positivos

# palabras de uso comun de los suscriptores, para solicitar o reportar informacion
# acerca del pago

palab_clase1 = ["pago", 'cuanto', 'transfe', 'transferencia']

palab_clase2 = ["mes", "mensualidad", "interne", "wifi"]


# formamos pares de palabras claves
palab_clase = form_pares(palab_clase1, palab_clase2)

var1 = mensajes.index[mensajes["Texto"].apply(lambda x: recore_pares(palab_clase, x))]

# aqui listado para depurar var1 
# generar celda para ejecutar la siguiente instruccion:
# mensajes.loc[mensajes.index[var1]]



In [52]:
# a var1 le quitamos var2, var3 y el resultado se clasifica como tipo 1 (en tipo_mensaje)
# hay q anotar que en mensajes tipo 1 se encuentran tambien los reportes de pago
# los cual implica que la respuesta automatica debe indicar "ademas" que se: 
# haga reporte por otra via o que espere para validar.

var2 = [3, 91, 151, 214, 443, 467, 481, 678]

var1 = sorted(list(set(var1) - set(var2)))

# se descartan los que ya fueron clasificados en anterior categoria

temp = mensajes.loc[mensajes.index[var1]]

# en este cruce se puede observar registros que ya fueron clasificados 
# el solapamiento se debe al contenido del mensaje
#print(tabcruz(temp.tipo_mensaje, temp.es_suscriptor, eti_tmensaje, eti_suscriptor))

var3 = temp.index[operator.and_(temp['es_suscriptor']==0, temp['tipo_mensaje']==0)]
var1 = sorted(list(set(var1) - set(var3)))

mensajes.loc[mensajes.index[var1], "tipo_mensaje" ] = 1
mensajes.loc[mensajes.index[var1], "es_suscriptor" ] = 0

In [53]:
# revisamos las frecuencias a ver q tal vamos

print(frecuencia(mensajes["es_suscriptor"], eti_suscriptor), "\n")

print(frecuencia(mensajes["tipo_mensaje"], eti_tmensaje), "\n")

# analizamos la tabulacion cruzada de las caracteristicas anteriores para
# comprobar la coherencia en los datos

print(tabcruz(mensajes.tipo_mensaje, mensajes.es_suscriptor, eti_tmensaje, eti_suscriptor))



col_0          frecuencia
es_suscriptor            
Si             360       
No             319       
Sin Clase      266        

col_0         frecuencia
tipo_mensaje            
falla Serv    190       
Info Pago     61        
Otros         319       
Sin Clase     375        

es_suscriptor   Si   No  Sin Clase
tipo_mensaje                      
falla Serv     190  0    0        
Info Pago      61   0    0        
Otros          0    319  0        
Sin Clase      109  0    266      


In [59]:
# ok, ahora nos concentramos en el subgrupo Si es sucriptor pero no esta
# clasificado en tipo de mensaje

var1 = mensajes.index[operator.and_(mensajes["es_suscriptor"]==0, mensajes["tipo_mensaje"]==9)]

# de este lote descartamos aquellos cuya tipo mensaje sea diferente de 1 (reporte pago)

var2 = [59, 101, 102, 103, 140, 144, 145, 146, 179, 242, 276, 277, 278, 279,
       290, 307, 309, 310, 313, 332, 362, 363, 364, 397, 398, 399, 401, 422,
       423, 424, 425, 429, 437, 445, 458, 459, 460, 494, 518, 519, 520, 553,
       571, 579, 581, 638, 643, 674, 675, 759, 805, 806, 807, 830, 837, 852, 
       858, 886, 897, 919]

var1 = sorted(list(set(var1) - set(var2)))

mensajes.loc[mensajes.index[var1], "tipo_mensaje"] = 1

# revisando el sigute listado se observa que todos pertenecen a cat 2 (otros)
#mensajes.loc[mensajes.index[var2]]
# var3 pertenecen a cat tipo_mensaje = 0
var3 = [101, 103, 144, 145, 242, 276, 290, 307, 313, 332, 397, 398,
       399, 422, 458, 459, 494, 579, 581, 830, 837, 852, 897]

var2 = sorted(list(set(var2) - set(var3)))

mensajes.loc[mensajes.index[var2], "tipo_mensaje"] = 2
mensajes.loc[mensajes.index[var3], "tipo_mensaje"] = 0

In [61]:
# analizamos nuevamente la tabulacion cruzada de las caracteristicas anteriores para
# comprobar la coherencia en los datos

print(tabcruz(mensajes.tipo_mensaje, mensajes.es_suscriptor, eti_tmensaje, eti_suscriptor))



es_suscriptor   Si   No  Sin Clase
tipo_mensaje                      
falla Serv     213  0    0        
Info Pago      110  0    0        
Otros          37   319  0        
Sin Clase      0    0    266      


In [2]:
mensajes = pd.read_pickle(path_datos+"mensajes.pkl")
# aqui voy 13-01-19 5:26 am
#mensajes.loc[operator.and_(mensajes["tipo_mensaje"]==9, mensajes["es_suscriptor"]==9)]

In [5]:
# visto el listado anterior, se establece la siguiente estrategia: gererar un dataframe
# a partir de este listado para aplicarle los filtros de las funciones anteriores.
# la hipotesis de trabajo de que algunas palabras son propias del lexico usado por
# los suscriptores en sus mensajes.

mensgesx99 = mensajes.loc[operator.and_(mensajes["tipo_mensaje"]==9, 
                                        mensajes["es_suscriptor"]==9)]


clase1 = ['luis', 'precio', 'pagar', 'no']

clase2 = ['transferencia', 'internet', 'sirve']

# formamos pares de palabras claves
palab_clase = form_pares(clase1, clase2)

var1 = mensgesx99.index[mensgesx99["Texto"].apply(lambda x: recore_pares(palab_clase, x))]

# aqui listado para depurar var1 
# generar celda para ejecutar la siguiente instruccion:
mensajes.loc[mensajes.index[var1]]



,Enviado/por,Fecha/Hora,Texto,ts,tem,es_suscriptor,tipo_mensaje
13,04248636635,2018-08-31 07:31:37,buenos dias sr luis si activo el internet,27097,49282.0,9,9
17,04248636635,2018-08-31 15:54:38,sr luis me.dice.su nombre.co.pleto q tengo varios luis aca agregados.xfa para la transferencia acabo de llegar a casa,57278,21068.0,9,9
31,04247774757,2018-03-09 09:18:16,buenos dias senor luis es betty del bloque 8 queria preguntarle sime puede chequear el internet del apto que no lo se conectar,33496,14579.0,9,9
49,04248636635,2018-01-09 17:09:02,sr luis ud me volvio a kitar el internet ayer la paguina del banco venezuela no me keria abrir para transferorle xeso no se la ise iva ir orota aserla yme kitaste de.nuevo el intern,61742,5463.0,9,9
65,04247345175,2018-03-09 19:25:18,senor luis ya le pague el internet para q me lo instale gracias,69918,1325.0,9,9
66,04247345175,2018-03-09 19:25:33,senor luis ya le pague el internet para q me lo instale gracias,69933,15.0,9,9
92,04164947520,2018-07-09 19:19:47,buenas noche sr. luis soy jhonattan aqui donde la sra. sublay si pudo actualizar el internet es para utilizar el internet.,69587,39469.0,9,9
99,04263261041,2018-07-09 12:29:45,hola en q precio esta el internet para cancelar,44985,9716.0,9,9
152,04247441060,2018-12-09 18:36:52,buenas tardes. amigo que esta pasando con el internet que hace dias que no sirve. bloque 19 apto 00 01,67012,17092.0,9,9
176,04164751779,2018-09-13 17:09:54,sr luis es la sra de la casa grande ya le hice la traferencia de los cable de la compujadora gracia,61794,428.0,9,9


In [9]:
no_suscript_en_var1 = [31, 176]

tmsje_0_en_var1 = [152, 180, 247, 248, 271, 272, 341, 407, 491, 803, 831, 834, 861, 905]

tmsje_2_en_var1 = [31, 176]

suscript_en_var1 = sorted(list(set(var1) - set(no_suscript_en_var1)))

mensajes.loc[mensajes.index[suscript_en_var1], 'es_suscriptor'] = 0
mensajes.loc[mensajes.index[no_suscript_en_var1], 'es_suscriptor'] = 1

tmsje_1_en_var1 = sorted(list(set(var1) - set(tmsje_0_en_var1) - set(tmsje_2_en_var1)))

mensajes.loc[mensajes.index[tmsje_0_en_var1], 'tipo_mensaje'] = 0
mensajes.loc[mensajes.index[tmsje_1_en_var1], 'tipo_mensaje'] = 1
mensajes.loc[mensajes.index[tmsje_2_en_var1], 'tipo_mensaje'] = 2

In [11]:
# analizamos nuevamente la tabulacion cruzada de las caracteristicas anteriores para
# comprobar la coherencia en los datos

print(tabcruz(mensajes.tipo_mensaje, mensajes.es_suscriptor, eti_tmensaje, eti_suscriptor))


es_suscriptor   Si   No  Sin Clase
tipo_mensaje                      
falla Serv     227  0    0        
Info Pago      137  0    0        
Otros          37   321  0        
Sin Clase      0    0    223      


In [5]:
mensajes = pd.read_pickle(path_datos+"mensajes.pkl")
#mensajes.loc[operator.and_(mensajes["tipo_mensaje"]==9, mensajes["es_suscriptor"]==9)]

#dir_tlf_suscrip = sorted(list(set(mensajes['Enviado/por'][mensajes.es_suscriptor==0])))

#mensajes.loc[mensajes["Enviado/por"].isin(dir_tlf_suscrip), "es_suscriptor"] = 0 

print(tabcruz(mensajes.tipo_mensaje, mensajes.es_suscriptor, eti_tmensaje, eti_suscriptor))


es_suscriptor   Si   No  Total
tipo_mensaje                  
falla Serv     274  0    274  
Info Pago      183  0    183  
Otros          104  384  488  
Total          561  384  945  


In [12]:
#mensajes = pd.read_pickle(path_datos+"mensajes.pkl")

#mensajes.loc[mensajes["Enviado/por"] == '04261749423', "es_suscriptor"] = 1 
#mensajes.loc[mensajes["Enviado/por"] == '04261749423', "tipo_mensaje"] = 2

var = mensajes.index[operator.and_(mensajes["tipo_mensaje"]==9, mensajes["es_suscriptor"]==0)]

tranferypagos = [15, 16, 23, 50, 70, 71, 100, 169, 175, 292, 335, 366, 368, 373, \
                374, 408, 409, 410, 419, 430, 431, 433, 501, 504, 507, 510,511, \
                513, 514, 530, 564, 594, 676, 684, 695, 717, 718, 742, 746, 747, \
                749, 815, 862, 902, 903, 906, 942] 


otros = [63, 64, 68, 177, 181, 214, 224, 253, 300, 325, 333, 334,351, 359, 372,
        375, 376, 406, 432, 467, 496, 508,528, 559, 607, 608, 614, 633, 634,
        641, 657, 658, 659, 660, 661, 662, 667, 687, 697, 702, 722, 725, 726,
        727, 728, 729, 730, 731, 753, 754, 770, 777, 787, 789, 790, 792, 793,
        794, 795, 799, 812, 816, 817, 869, 870, 871, 876]

fallas = sorted(list(set(var)-set(tranferypagos)-set(otros)))

mensajes.loc[mensajes.index[fallas], "tipo_mensaje"] = 0
mensajes.loc[mensajes.index[tranferypagos], "tipo_mensaje"] = 1
mensajes.loc[mensajes.index[otros], "tipo_mensaje"] = 2

mensajes.loc[mensajes.index[var]]

# una vez conforme con resultados se guarda 
mensajes.to_pickle(path_datos+"mensajes.pkl")

#mensajes.loc[mensajes["Enviado/por"]=='04147243342']

In [13]:
mensgesx99 = mensajes.loc[operator.and_(mensajes["tipo_mensaje"]==9, mensajes["es_suscriptor"]==9)]

frec = pd.crosstab(index=mensgesx99["Enviado/por"], columns = "frecuencia")

frec.sort_values('frecuencia', ascending = False, inplace = True)

frec

col_0,frecuencia
Enviado/por,
04247620671,8
04164793931,6
04265530194,5
04247028750,5
04247586731,4
04147103139,3
04160752385,3
04161319757,3
04162122522,2


In [42]:
var = mensajes.index[mensajes['Enviado/por']=='04247620671']
mensajes.loc[mensajes.index[var], ['es_suscriptor', 'tipo_mensaje']] = [1, 2]

In [32]:
print(tabcruz(mensajes.tipo_mensaje, mensajes.es_suscriptor, eti_tmensaje, eti_suscriptor))

es_suscriptor   Si   No  Sin Clase
tipo_mensaje                      
falla Serv     223  0    0        
Info Pago      139  0    0        
Otros          52   340  0        
Sin Clase      157  0    79       


In [44]:
for telef in frec.index:
    print(telef)

04247620671
04164793931
04169298551
04265530194
04247028750
04247586731
04147103139
04147154451
04160752385
04161319757
04121025191
04162122522
04267293055
04265780504
04247178935
04163426930
04164751779
04147394017
04127636359
04141863535
04149765459
04161981823
04263792638
04269282259
04268021012
04122490064
04265786016
04126907553
04264662020
04127884440
04261784384
04247427411
04143760380
04145320217
04147243342
04168764227
04166744097
04121084834
04269790661


In [1]:
from ipywidgets import widgets
from IPython.display import display
text = widgets.Text()
display(text)

def handle_submit(sender):
    print(text.value)

text.on_submit(handle_submit)

Text(value='')

hola!


In [2]:
boton = widgets.Button(description='Tocala!')
display(boton)

def en_boton_tocado(b):
    print('Boton tocado..')

boton.on_click(en_boton_tocado)

Button(description='Tocala!', style=ButtonStyle())

Boton tocado..


In [3]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def f(x):
    print(x)
interact(f, x=var)

NameError: name 'var' is not defined

**es_suscriptor: esta variable toma valor 0 si el contenido de "Texto" proviene de un
cliente de la red comunitaria, y 1 en caso contrario. Pero ¿Como se determina si el 
mensaje proviene o no de un suscriptor?**
**a vuelo de pajaro podemos decir que:**

**1- Si hacemos query de "Enviado/por" en la base de datos de cliente y aparece en esta
definitivamente el mensaje fue enviado por un suscriptor, es el caso mas sencillo**

**2- Que pasa si el query no devuelve datos, quiere decir que no se encuentra registrado
pero esto no quiere decir que el mensaje no provenga de un cliente, sino que tal vez
este usando otro dispositivo no registrado en la base de datos, de alli que surga la necesidad
de analizar el contenido para establecer el valor de la variable es_suscriptor.
en este apartado se puede inferir que tal vez el usuario se identifique con su nombre
dentro del contenido del mensaje o tal vez no, en caso de que si, la solucion pasaria algo
similar a 1, en caso contrario se necesitara establecer mas precision en cuanto al contenido
de "Texto" y hasta sera necesario interpretar varios mensajes simultaneamente del mismo 
Enviado/por" en un tiempo-estipulado e interactuar con el remintente para clasificarlo.**

**3- Otro aspecto que pareciera ser prometedor es la frecuencia de origen de sms, obtenida de 
la variable "Enviado/por"**



**Bien, repasando de nuevo, un mensaje se considera que proviene de un suscriptor en estos 
casos:**

**1- su origen plasmado en la variable "Enviado/por", resuelto anteriormente**

**2- contenga frases asociadas con el servicio de internet, en especial las siguientes
son trascendentes: "no hay conexion", "no hay internet", "cuanto es la mensualidad"**

**3- tomar en cuenta comodin por error ortografico comun "interne" y otros**


In [2]:
# Load dataframe from pickled pandas object
import pandas as pd

path_datos = '/home/luis/mensajes.data/'

datos = pd.read_pickle(path_datos+"mensajes.pkl")

datos.head()

,Enviado/por,Fecha/Hora,Texto,ts,tem,es_suscriptor,tipo_mensaje
0,04140819214,2018-08-29 16:31:19,luis que si le mando la plata de bicentimario ...,59479,NaN,1,2
1,04140819214,2018-08-30 08:34:25,ps luis qe si mamdo la plata que esta en mi cu...,30865,28614.0,1,2
2,04145328570,2018-08-28 18:03:12,hola dios te bendiga no hay internet,64992,34127.0,0,0
3,04145328570,2018-01-09 10:15:48,hola senor luis buenas tarde es para decierle ...,36948,28051.0,0,0
4,04145328570,2018-01-09 10:16:09,puede verificar por mi numero de cedula 979164...,36969,21.0,0,1


In [2]:
datos.shape

(945, 7)

In [7]:
for columna in datos.columns:
    print(columna)

Enviado/por
Fecha/Hora
Texto
ts
tem
es_suscriptor
tipo_mensaje


In [8]:
var = tabcruz(datos.tipo_mensaje, datos.es_suscriptor, eti_tmensaje, eti_suscriptor)

In [9]:
var

es_suscriptor,Si,No,Total
tipo_mensaje,,,
falla Serv,274,0,274
Info Pago,183,0,183
Otros,104,384,488
Total,561,384,945


In [10]:
type(var)

pandas.core.frame.DataFrame

In [16]:
a = datos.loc[datos['tipo_mensaje']==2]
b = a.loc[a['es_suscriptor']==0]['Texto'].values

In [18]:
for mensaje in b:
    print(mensaje)

donde andas senor luis
a ok senor luis ta bm
a ok senor luis ta bn
hola ayer me  dijo que 50 soberanos  porfa  nombre weky  clave ferbrujjsa
sr luis ya vamos para la casa 
ok gracias de luis 
listi hermano erws un genio
gracia a usted y disculpe la tradancia
sra yhovanna del 19
ok lo que pasa es que estoy trabajando en la casa no hay nadie
es la sra yhovanna
buen dia amigo sera que tiene chance de revisar el router
hola sr luis
hola buenas tarde sr luis soy blocdoon mi hija me dijo que le recordara como a las 4:00 o 4:30 para usted venir a mi casa
 ah ok y cuanto cobra usted por eso 
hola buenos dias sr luis cuanto vale el mantenimiento del cpu 
ah bien y que velocidad tendria pues me llega bien tu senal y ahi fue donde vi el numero.
sr luis yo tengo un bit nuevo acceso de banda ancha eso no sirve como router 
 ok se luis le escribo como a la nueve haber si esta en su negocio para ir 
sr luis a que hora abre en la tarde
hola corazon soy marithza
hola corazon soy marithza
buen dia ok gr